# List of My Movies

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-30 14:49:39.223755


In [2]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            location = moviesMine[result[0]]
            print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                location = moviesMine[result[0]]
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
        
        
    def mergeMovies(self, toFile=False, movie=None, source=None, year=None):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")
        
        yearcut = year
        sourcecut = source
        moviecut = movie

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        yearRenames = {}
        for key,name in moviesCombined.items():
            movie   = key[:-7]
            year = key[-5:-1]
            
            if yearcut is not None:
                if int(year) not in [int(yearcut)-1,int(yearcut),int(yearcut)+1]:
                    continue
            if sourcecut is not None:
                if name != sourcecut:
                    continue
            if moviecut is not None:
                if moviecut not in movie:
                    continue

            location = moviesMine.get(key)
            if location is None:
                for dYear in [-1,1,-2,2]:
                    dKey = "{0} [{1}]".format(movie, int(year)+dYear)
                    if moviesMine.get(dKey) is not None:
                        location = moviesMine[dKey]
                        break
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)

                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]

                    yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]

                        yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("\n")
                for key,rename in yearRenames.items():
                    if key != rename:
                        print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)".format(key, rename))                
                
                print("\n")
                yearRenames = {}
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

In [3]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Wed Jan 30, 2019 14:49:40 for Last Run


In [5]:
mine.suggestRenames()

In [6]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5249 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 425.1kB.
Current Time is Wed Jan 30, 2019 14:50:20 for Last Run


In [12]:
mine.searchMyMovies("The Nutcracker", 2009, cutoff=0.6)

The Nutcracker  --->  ['The Nutcracker [2010]']
moveFile(src="/Volumes/Download/Movies/The Nutcracker [2010].mp4", dst="/Volumes/Download/Movies/The Nutcracker [2010].mp4", debug=True)



In [13]:
mine.mergeMovies(movie="Mission")

Year  ✓  Movie                                                                                               Source              
1966     Mission Of Fear (Astataïon, ou Le Festin des morts)                                                 Canada              




1982     The Missionary                                                                                      Box Office Mojo     




1986  ✓  The Mission                                                                                         Oscar               




1996  ✓  Mission: Impossible                                                                                 Box Office Mojo     




2000  ✓  Mission Impossible 2                                                                                Box Office Mojo     
2000  ✓  Mission To Mars                                                                                     Box Office Mojo     




2006  ✓  Mission: Impossible III                                      

In [34]:

moveFile(src="/Volumes/Download/Movies/Star Wars The Last Jedi [2017].mp4", dst="/Volumes/Download/Movies/Star Wars: Episode VIII - The Last Jedi.mp4")